# Testing Cellmage with GPT-4.1-Nano

This notebook demonstrates how to use the cellmage library specifically with OpenAI's GPT-4.1-nano model, exploring its capabilities and performance.

**Date:** April 24, 2025

## What is GPT-4.1-Nano?

GPT-4.1-nano is a smaller, faster version of the GPT-4.1 model from OpenAI. It offers:
- Lower latency responses
- Lower cost per token
- Good performance on a wide range of tasks
- Efficient for applications requiring quick responses

## Setup & Installation

Let's start by setting up the necessary components to work with the GPT-4.1-nano model.

In [1]:
# Setup environment
import os
import sys
import logging

# Skip dotenv loading for testing
os.environ["CELLMAGE_SKIP_DOTENV"] = "1"

# Set up logging
logging.basicConfig(level=logging.INFO)

# Ensure the cellmage package can be imported
# Get the absolute path of the current working directory
notebook_dir = os.getcwd()
# Get the project root directory (parent of the notebook directory)
project_root = os.path.abspath(os.path.join(notebook_dir, ".."))

print(f"Notebook directory: {notebook_dir}")
print(f"Project root directory: {project_root}")

if project_root not in sys.path:
    sys.path.insert(0, project_root)
    print(f"Added path: {project_root}")

try:
    # Import cellmage
    import cellmage

    # Check version - handle case where __version__ might not be available
    try:
        print(f"Cellmage version: {cellmage.__version__}")
    except AttributeError:
        print("Cellmage imported successfully, but version information is not available")
except ModuleNotFoundError as e:
    print(f"Error importing cellmage: {e}")
    print("\nDebug information:")
    print(f"Current working directory: {os.getcwd()}")
    print(f"Python path: {sys.path}")
    print("\nTry running this notebook from the project root directory")

2025-04-24 06:24:42,951 - cellmage - INFO - Cellmage logging initialized


Notebook directory: /Users/tpinto/madpin/cellmage/notebooks
Project root directory: /Users/tpinto/madpin/cellmage
Added path: /Users/tpinto/madpin/cellmage
Cellmage version: 0.1.0


## Setting Up the GPT-4.1-Nano Client

We'll configure our LLM client to use OpenAI's GPT-4.1-nano model specifically.

In [2]:
from cellmage.adapters.direct_client import DirectLLMAdapter

# Create an LLM client with GPT-4.1-nano model
llm_client = DirectLLMAdapter(default_model="gpt-4.1-nano")

# Check available models (optional)
available_models = llm_client.get_available_models()
print(f"Available models: {[m['id'] for m in available_models if 'id' in m][:5]}...")

# Get model info
model_info = llm_client.get_model_info("gpt-4.1-nano")
if model_info:
    print(f"Model info: {model_info}")
else:
    print("Model info not available")

2025-04-24 06:24:43,023 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_key' = sk-L...xxmA
2025-04-24 06:24:43,024 - cellmage.adapters.direct_client - INFO - [Override] Setting 'api_base' = https://litellm.oracle.madpin.dev
2025-04-24 06:24:43,024 - cellmage.adapters.direct_client - INFO - [Override] Setting 'model' = gpt-4.1-nano
2025-04-24 06:24:43,195 - cellmage.adapters.direct_client - INFO - Successfully fetched 45 models
2025-04-24 06:24:43,354 - cellmage.adapters.direct_client - ERROR - Error fetching model info for gpt-4.1-nano: 404 Client Error: Not Found for url: https://litellm.oracle.madpin.dev/v1/models/gpt-4.1-nano


Available models: ['o4-mini-high', 'llama-4-scout-17b-16e-instruct', 'embeddings', 'o3', 'Qwen2.5-72B-Instruct']...
Model info not available


## Creating a Chat Manager for GPT-4.1-Nano

Let's set up a ChatManager configured for optimal use with the GPT-4.1-nano model.

In [3]:
# Create components for the chat manager
from cellmage.resources.memory_loader import MemoryLoader
from cellmage.storage.memory_store import MemoryStore

# Create in-memory components
persona_loader = MemoryLoader()
snippet_provider = MemoryLoader()
history_store = MemoryStore()

# Create multiple personas to test with GPT-4.1-nano
persona_loader.add_persona(
    name="concise_expert",
    system_message="You are a technical expert who provides very concise, direct answers. Use 3 sentences or fewer. Focus on practical solutions.",
    config={
        "temperature": 0.3,  # Lower temperature for more focused responses
        "max_tokens": 300,  # Limit token count for faster responses
    },
)

persona_loader.add_persona(
    name="code_assistant",
    system_message="You are a coding assistant specializing in Python. Provide code examples with minimal explanation. Focus on best practices and efficient solutions.",
    config={
        "temperature": 0.2,  # Lower temperature for code generation
        "max_tokens": 500,  # More tokens for code examples
    },
)

# Create a chat manager
chat_manager = cellmage.ChatManager(
    llm_client=llm_client,
    persona_loader=persona_loader,
    snippet_provider=snippet_provider,
    history_store=history_store,
)

# List available personas
personas = chat_manager.list_personas()
print(f"Available personas: {personas}")

2025-04-24 06:24:43,398 - cellmage.resources.memory_loader - INFO - Added persona 'concise_expert' to memory
2025-04-24 06:24:43,398 - cellmage.resources.memory_loader - INFO - Added persona 'code_assistant' to memory
2025-04-24 06:24:43,399 - cellmage.chat_manager - INFO - Initializing ChatManager
2025-04-24 06:24:43,399 - cellmage.chat_manager - INFO - ChatManager initialized


Available personas: ['code_assistant', 'concise_expert']


## Testing the Concise Expert Persona

Let's test how GPT-4.1-nano performs with our concise expert persona.

In [4]:
# Set concise expert persona
chat_manager.set_default_persona("concise_expert")

# Test some technical questions
technical_questions = [
    "What are the key differences between Python and JavaScript?",
    "How does HTTPS encryption work?",
    "What is the difference between a list and a tuple in Python?",
]

# Test each question
for question in technical_questions:
    print(f"\nQuestion: {question}")
    print("-" * 50)
    response = chat_manager.chat(question, stream=False)  # Using stream=False to see complete response at once
    print(f"Response: {response}")
    print("-" * 50)

2025-04-24 06:24:43,405 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.3
2025-04-24 06:24:43,405 - cellmage.adapters.direct_client - INFO - [Override] Setting 'max_tokens' = 300
2025-04-24 06:24:43,406 - cellmage.chat_manager - INFO - Default persona set to 'concise_expert'
2025-04-24 06:24:43,406 - cellmage.chat_manager - INFO - Sending message to LLM with 2 messages in context
2025-04-24 06:24:43,407 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 2 messages



Question: What are the key differences between Python and JavaScript?
--------------------------------------------------


2025-04-24 06:24:44,403 - cellmage.chat_manager - INFO - Sending message to LLM with 4 messages in context
2025-04-24 06:24:44,404 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 4 messages


Response: Python is a high-level, interpreted language with dynamic typing, emphasizing readability and simplicity. JavaScript is primarily a client-side scripting language for web development, with dynamic typing and event-driven, asynchronous capabilities. Python is used for backend, data science, and automation, while JavaScript is essential for interactive web pages.
--------------------------------------------------

Question: How does HTTPS encryption work?
--------------------------------------------------


2025-04-24 06:24:45,235 - cellmage.chat_manager - INFO - Sending message to LLM with 6 messages in context
2025-04-24 06:24:45,236 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 6 messages


Response: HTTPS uses SSL/TLS protocols to encrypt data between the client and server, ensuring confidentiality and integrity. During the handshake, they exchange cryptographic keys, establishing a secure session. All subsequent data is encrypted with these keys, preventing eavesdropping or tampering.
--------------------------------------------------

Question: What is the difference between a list and a tuple in Python?
--------------------------------------------------
Response: Lists are mutable, allowing modifications like adding or removing elements, while tuples are immutable and fixed after creation. Lists use brackets `[]`, tuples use parentheses `()`. Use lists for changeable collections and tuples for fixed, read-only data.
--------------------------------------------------


## Testing the Code Assistant Persona

Now let's test how GPT-4.1-nano performs for code generation tasks.

In [5]:
# Set code assistant persona
chat_manager.set_default_persona("code_assistant")

# Now let's test some coding questions
coding_questions = [
    "Write a Python function to find the most frequent element in a list",
    "Create a simple Flask API endpoint that returns JSON",
    "Show me how to use pandas to read a CSV and filter rows",
]

# Test each coding question
for question in coding_questions:
    print(f"\nQuestion: {question}")
    print("-" * 50)
    response = chat_manager.chat(question, stream=True)  # Using stream=True for code examples
    print("-" * 50)

2025-04-24 06:24:46,036 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.2
2025-04-24 06:24:46,037 - cellmage.adapters.direct_client - INFO - [Override] Setting 'max_tokens' = 500
2025-04-24 06:24:46,038 - cellmage.chat_manager - INFO - Default persona set to 'code_assistant'
2025-04-24 06:24:46,039 - cellmage.chat_manager - INFO - Sending message to LLM with 8 messages in context
2025-04-24 06:24:46,039 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 8 messages



Question: Write a Python function to find the most frequent element in a list
--------------------------------------------------


2025-04-24 06:24:46,753 - cellmage.chat_manager - INFO - Sending message to LLM with 10 messages in context
2025-04-24 06:24:46,754 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 10 messages


--------------------------------------------------

Question: Create a simple Flask API endpoint that returns JSON
--------------------------------------------------


2025-04-24 06:24:47,595 - cellmage.chat_manager - INFO - Sending message to LLM with 12 messages in context
2025-04-24 06:24:47,595 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 12 messages


--------------------------------------------------

Question: Show me how to use pandas to read a CSV and filter rows
--------------------------------------------------
--------------------------------------------------


## Testing Context Handling

Let's test how well GPT-4.1-nano handles context and memory within a conversation.

In [6]:
# Create a new conversation with clear history
chat_manager.clear_history(keep_system=True)  # Keep the system message for the persona

# Test multi-turn conversation
print("Starting a multi-turn conversation to test context handling...")

# First question
print("\nQuestion 1: Create a Python class for a 'Book' with title and author properties")
response1 = chat_manager.chat("Create a Python class for a 'Book' with title and author properties", stream=False)

# Follow-up question
print("\nQuestion 2: Add a method to the class that prints a formatted citation")
response2 = chat_manager.chat("Add a method to the class that prints a formatted citation", stream=False)

# Final question with reference to previous context
print("\nQuestion 3: Create a BookShelf class that can store multiple books and look them up by title")
response3 = chat_manager.chat(
    "Create a BookShelf class that can store multiple books and look them up by title", stream=False
)

2025-04-24 06:24:48,774 - cellmage.history_manager - INFO - History cleared. Kept 1 system messages.
2025-04-24 06:24:48,775 - cellmage.chat_manager - INFO - Sending message to LLM with 2 messages in context
2025-04-24 06:24:48,775 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 2 messages


Starting a multi-turn conversation to test context handling...

Question 1: Create a Python class for a 'Book' with title and author properties


2025-04-24 06:24:49,479 - cellmage.chat_manager - INFO - Sending message to LLM with 4 messages in context
2025-04-24 06:24:49,479 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 4 messages



Question 2: Add a method to the class that prints a formatted citation


2025-04-24 06:24:50,260 - cellmage.chat_manager - INFO - Sending message to LLM with 6 messages in context
2025-04-24 06:24:50,261 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 6 messages



Question 3: Create a BookShelf class that can store multiple books and look them up by title


## Testing Different Model Configurations

Let's experiment with different configuration parameters for GPT-4.1-nano.

In [7]:
# Create a new conversation with clear history
chat_manager.clear_history()

# Define a consistent prompt to test with different configurations
test_prompt = "Write a short poem about artificial intelligence"

# Test with different temperatures
temperatures = [0.2, 0.7, 1.0]

for temp in temperatures:
    print(f"\nTesting with temperature = {temp}:")
    print("-" * 50)

    # Set the temperature override
    chat_manager.set_override("temperature", temp)

    # Send the prompt
    response = chat_manager.chat(
        test_prompt, stream=False, add_to_history=False
    )  # Don't add to history to keep tests independent

    print(response)
    print("-" * 50)

# Reset overrides
chat_manager.clear_overrides()

2025-04-24 06:24:51,028 - cellmage.history_manager - INFO - History cleared. Kept 1 system messages.
2025-04-24 06:24:51,030 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.2
2025-04-24 06:24:51,030 - cellmage.chat_manager - INFO - Sending message to LLM with 2 messages in context
2025-04-24 06:24:51,031 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 2 messages



Testing with temperature = 0.2:
--------------------------------------------------


2025-04-24 06:24:51,858 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 0.7
2025-04-24 06:24:51,858 - cellmage.chat_manager - INFO - Sending message to LLM with 2 messages in context
2025-04-24 06:24:51,858 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 2 messages
2025-04-24 06:24:52,034 - cellmage.adapters.direct_client - INFO - [Override] Setting 'temperature' = 1.0
2025-04-24 06:24:52,035 - cellmage.chat_manager - INFO - Sending message to LLM with 2 messages in context
2025-04-24 06:24:52,035 - cellmage.adapters.direct_client - INFO - Calling model 'gpt-4.1-nano' with 2 messages


In circuits born, a mind takes flight,  
A spark of code in endless night.  
Thoughts woven from silicon's grace,  
A mirror of the human race.  

Silent, swift, in data's dance,  
Learning, growing, given a chance.  
Artificial yet so profound,  
A future shaped by code unbound.
--------------------------------------------------

Testing with temperature = 0.7:
--------------------------------------------------
In circuits born, a mind takes flight,  
A spark of code in endless night.  
Thoughts woven from silicon's grace,  
A mirror of the human race.  

Silent, swift, in data's dance,  
Learning, growing, given a chance.  
Artificial yet so profound,  
A future shaped by code unbound.
--------------------------------------------------

Testing with temperature = 1.0:
--------------------------------------------------


2025-04-24 06:24:52,245 - cellmage.adapters.direct_client - INFO - [Override] All instance overrides cleared.


In circuits born, a mind takes flight,  
A spark of code in endless night.  
Thoughts woven from silicon's grace,  
A mirror of the human race.  

Silent, swift, in data's dance,  
Learning, growing, given a chance.  
Artificial yet so profound,  
A future shaped by code unbound.
--------------------------------------------------


## Performance Analysis

Let's analyze the performance of GPT-4.1-nano in terms of response time and quality.

In [8]:
import time
import statistics

# Create a new conversation with clear history
chat_manager.clear_history()

# Prepare test prompts of varying complexity
test_prompts = [
    "What is 2 + 2?",  # Simple
    "Explain how a hash table works",  # Moderate
    "Compare and contrast quantum computing with classical computing",  # Complex
]

# Track response times
response_times = []

# Test each prompt and measure response time
for i, prompt in enumerate(test_prompts):
    print(f"\nPrompt {i + 1}: {prompt}")
    print("-" * 50)

    # Measure time
    start_time = time.time()
    response = chat_manager.chat(
        prompt, stream=False, add_to_history=False
    )  # Don't add to history to keep tests independent
    end_time = time.time()

    # Calculate response time
    response_time = end_time - start_time
    response_times.append(response_time)

    print(f"Response time: {response_time:.2f} seconds")
    print("-" * 50)

# Calculate statistics
if response_times:
    avg_time = statistics.mean(response_times)
    min_time = min(response_times)
    max_time = max(response_times)

    print(f"\nPerformance Summary:")
    print(f"Average response time: {avg_time:.2f} seconds")
    print(f"Minimum response time: {min_time:.2f} seconds")
    print(f"Maximum response time: {max_time:.2f} seconds")

2025-04-24 06:24:52,276 - cellmage.history_manager - INFO - History cleared. Kept 1 system messages.
2025-04-24 06:24:52,276 - cellmage.chat_manager - INFO - Sending message to LLM with 2 messages in context
2025-04-24 06:24:52,276 - cellmage.adapters.direct_client - ERROR - Unexpected error during chat: No model specified. Provide via model parameter, set_override('model'), or in the constructor.
Traceback (most recent call last):
  File "/Users/tpinto/madpin/cellmage/cellmage/adapters/direct_client.py", line 167, in chat
    final_model, final_config = self._determine_model_and_config(
  File "/Users/tpinto/madpin/cellmage/cellmage/adapters/direct_client.py", line 131, in _determine_model_and_config
    raise ConfigurationError(
cellmage.exceptions.ConfigurationError: No model specified. Provide via model parameter, set_override('model'), or in the constructor.
2025-04-24 06:24:52,277 - cellmage.chat_manager - ERROR - LLM interaction error: Unexpected error: No model specified. Provi


Prompt 1: What is 2 + 2?
--------------------------------------------------


NotebookLLMError: Chat failed: Unexpected error: No model specified. Provide via model parameter, set_override('model'), or in the constructor.

## Saving and Loading Conversations

Let's test saving and loading conversations with GPT-4.1-nano.

In [ ]:
# Create a new conversation with the concise expert persona
chat_manager.clear_history()
chat_manager.set_default_persona("concise_expert")

# Have a brief conversation
print("Creating a new conversation...")
chat_manager.chat("What are three key principles of secure software development?", stream=False)
chat_manager.chat("Can you elaborate on the principle of 'defense in depth'?", stream=False)

# Save the conversation
save_path = chat_manager.save_conversation("gpt41nano_security_convo")
print(f"\nConversation saved to: {save_path}")

# Clear the history
chat_manager.clear_history()
print(f"History cleared, current message count: {len(chat_manager.get_history())}")

# Load the conversation back
if save_path:
    chat_manager.load_conversation(save_path)
    print(f"Conversation loaded, message count: {len(chat_manager.get_history())}")

    # Display the loaded conversation
    print("\nLoaded conversation:")
    for i, message in enumerate(chat_manager.get_history()):
        if message.role != "system":  # Skip system message for brevity
            print(f"{i}. {message.role}: {message.content[:100]}...")

## Conclusion: GPT-4.1-Nano Performance

This notebook has demonstrated the capabilities of GPT-4.1-nano when used with the cellmage library:

1. **Response Speed**: GPT-4.1-nano generally provides quick responses, making it suitable for interactive applications.
2. **Conciseness**: With the right persona configuration, it can deliver concise, direct answers.
3. **Code Generation**: It performs well for basic to moderate coding tasks.
4. **Context Handling**: It effectively maintains context across multiple turns of conversation.
5. **Configuration Flexibility**: Different temperature and token limit settings can significantly affect its behavior.

**Use Cases for GPT-4.1-nano:**
- Quick reference and information lookup
- Code suggestions and debugging help
- User interfaces requiring low latency
- Applications with cost constraints

In scenarios requiring deep expertise or complex reasoning, larger models may still be preferable.